In [13]:
# Obsolete
from config import setup
setup()

 # Projet AP GeoSafe, Evacuation lors de feu
 
 ## Generator 
 Pour générer les données utilisées après par le projet, il faut utiliser le générateur du projet evacsim

In [1]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
import numpy as np
import math

configuration

In [2]:
from config_duc import setup
setup()

platform:  linux


## modélisation

In [17]:
def geosafe_model(l, q, eps, r, W, H, d):
    '''       
    Int  Matrix l: length path from i to j = G.l[i,j]. No path ij <--> G.l[i,j] = 0
    Int  Array  q: capacity of node i <--> G.q[i]
    Bool Array  eps: evacuation node 
    Bool Array  r: safe node
    Int  Array  W: initial population at node i <--> W[i]
    Int         H : Time span
    Int  Array  d: deadline to leave the node
    '''
    
    nb_node = q.shape[0]
    nodes = np.arange(nb_node) # id nodes, i.e [1,2,3,4, ...]
    total_population = np.sum(W)

    mdl = CpoModel(name='geosafe')
    
    # == Output ==
    # starting date
    s = np.array( mdl.integer_var_list(nb_node, min=0, max = H, name="s") )
        
    # evacuation rate aka. height of package
#     h = np.array([
#         mdl.integer_var(name='phi[%d]')%(i+1), min=0, max = W[i] for i in range(nb_nodes)
#     ])
    h = np.array( mdl.integer_var_list(nb_node, min=0, max = 100, name="h") )

    # == Intermediate ==
    # node flow of population
    phi = np.matrix([
            [ 
                mdl.integer_var(name='phi[%d,%d]'%(i+1,j)) for j in range(H) 
            ] for i in range(nb_node)
        ])
    
    # ending date (leaving time) of node
    e = np.array( mdl.integer_var_list(nb_node, min=0, max = 150, name= "e") )
    
    for v in range(nb_node):
        if eps[v]:
            mdl.add(
                e[v] == s[v] + W[v]/h[v]
            )
    
    # == Constraints ==
    
    # Time long enough to evacuate (additional)
    for v in nodes[eps]:
        mdl.add( h[v] * H >= W[v] )
    
    # evacuate everyone in evacuation node (???)
    # has been moved above
    
    # Flow at a node u = sum from all of its leaves node epsilon
#     mdl.add(
#         phi[u, t] == np.sum( phi[u, t - l[u,v] - s[eps]] ) \
#             for t in range(H)                                  \
#             for v in np.where(eps)                           \
#             for u in np.where(t)                             
#     )

    # simplified version
#     for u in nodes[np.where(np.logical_and(np.invert(eps), np.invert(r)))]:
        
#         for t in range(H):
            
#             v_arrays = nodes[np.where(
#                                 np.logical_and(
#                                     np.logical_and(
#                                         np.logical_and(
#                                             eps,
#                                             l[:,u] > 0, 
#                                         ),
#                                         s + l[:,u] <= t,
#                                     ),
#                                     t <= e + l[:, u]
#                                 )
#                             )[1]]
            
#             mdl.add(
#                 phi[u, t] == mdl.sum(h[v] for v in v_arrays)
#             )

#     for u in nodes[eps]:
#         mdl.add(
#             mdl.sum(phi[u, t] for t in range(H)) == h[u]
#         )

    for t in range(H):

        # Flow at epsilon should be h during the evacutation period
        for u in nodes[eps]:
            mdl.add( 
                phi[u, t] == h[u]
            )
            
        
        # Flow at a node u = sum from all of its leaves node epsilon
        for u in nodes[np.where(np.logical_and(np.invert(eps), np.invert(r)))]:
            mdl.add(
                phi[u, t] == mdl.sum(h[v] for v in nodes[eps] if l[v,u] != math.inf and s[v] + l[v,u] <= t and t <= e[v] + l[v,u])
            )
    
    # Flow at a node does not excess capacity of arc
    for t in range(H):
        for u in nodes:
            mdl.add(
                phi[u,t] <= q[u]
            )
            
    # Objective
#     mdl.add(
#         minimize(mdl.max(s[eps] + h[eps]/W[eps] - d[eps]))
#     )
    mdl.add(
        minimize(mdl.max(s[u] + W[u]/h[u] - d[u] for u in nodes[eps] ))
    )
    
    return mdl

## Unit test:

- Test input

In [12]:
import networkx as nx

In [13]:
G = nx.DiGraph()

global_d = 250 # TODO: delete this one, just for debug

# TODO: Maybe we should use eps = W > 0, so that do not need to use 'eps' attribute?
nodes_state = [
    (1, {'eps': True,  'r': False, 'W': 5, 'q': 4, 'd': global_d}),
    (2, {'eps': True,  'r': False, 'W': 5, 'q': 3, 'd': global_d}),
    (3, {'eps': True,  'r': False, 'W': 5, 'q': 7, 'd': global_d}),
    (4, {'eps': False, 'r': False, 'W': 0, 'q': 3, 'd': global_d}),
    (5, {'eps': False, 'r': False, 'W': 0, 'q': 20, 'd': global_d}),
    (6, {'eps': False, 'r': True,  'W': 0, 'q': 100, 'd': global_d})
]

edges_state = [
    (1, 4, {'l' : 4}),
    (2, 4, {'l' : 3}),
    (4, 5, {'l' : 7}),
    (3, 5, {'l' : 3}),
    (5, 6, {'l' : 10})
]

G.add_nodes_from(nodes_state)
G.add_edges_from(edges_state)

Node Attribute :

In [14]:
def _node_attribute_(G, attribute='eps'):
    '''
    Author: Duc Hau :D
    Return a numpy boolean array of attribute
    '''
    values_tmp = nx.get_node_attributes(G, attribute).values()
    return np.array(list(values_tmp))

- Init model

In [15]:
# Path length
#l = nx.to_numpy_matrix(G, weight='l')
l = nx.floyd_warshall_numpy(G, weight='l')

# Node capacities
q = _node_attribute_(G, 'q')

# If the node is in EPSILON (evacuating node)
eps = _node_attribute_(G, 'eps')

# If the node is in the ROOT (safe node)
r = _node_attribute_(G, 'r')

# Initial population
W = _node_attribute_(G, 'W')

# Time span
H = 30

# Deadline for evacutation
d = _node_attribute_(G, 'd')

# Solver configuration
ctx = {}

In [18]:
mdl = geosafe_model(l, q, eps, r, W, H, d)

- solve model

In [19]:
mdl.solve()
sol = mdl.solve(Presolve='On', Workers='Auto')
print(sol.get_solver_log())

print('\n\n Result:')
if sol.is_solution():
    sol.print_solution()
else:
    print('Problem does not have solution')
    sol.print_solution()

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 189 variables, 336 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 1629.1 (before), 1629.1 (after)
 !  . Memory usage      : 414.2 kB (before), 414.2 kB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        189                 -
 + New bound is -247.500
                        0         41                 -
 + New bound is -247
                        0         41    1   F        -
 + New bound is -245.016
 *          -245       36  0.00s        1      (gap is 0.0064%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : -245 (optimal - effective tol. is 0.024